In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
import scipy
import string
from scipy.sparse import hstack
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import os
import re

In [2]:
import spacy

In [3]:
data = pd.read_csv("HRTM_CASE.csv", encoding = "ISO-8859-1")

In [100]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11125 entries, 0 to 11640
Data columns (total 5 columns):
KeyCategory           11125 non-null object
KeyServiceOffering    11125 non-null float64
Subject               11125 non-null object
Description           11125 non-null object
newText               11125 non-null object
dtypes: float64(1), object(4)
memory usage: 521.5+ KB


In [27]:
data.head()

,KeyCategory,KeyServiceOffering,Subject,Description
0,12.0,42.0,Change of Start Date for New Hire,"Hi Team,\n\nPlease kindly change the onboardin..."
1,5.0,18.0,NaN,NaN
2,5.0,18.0,NaN,NaN
3,4.0,12.0,Unable to change salary crediting account via ...,"Dear Sir/Madam,\n\nI tried to change the bank ..."
4,6.0,29.0,Inquiry About Health Check-up,"Hi,\n\nI have given blood samples for annual h..."


In [4]:
def removeNewLine(s):
    s = re.sub('\n', ' ', s)
    s = re.sub('/', ' ', s)
    s = re.sub('Hi', ' ', s)
    s = re.sub('Hello', ' ', s)
    return s

In [5]:
data['Description'] = data.apply(lambda x: str(x['Description']), axis = 1)
data['Subject'] = data.apply(lambda x: str(x['Subject']), axis = 1)
data['KeyCategory'] = data.apply(lambda x: str(x['KeyCategory']), axis = 1)
data['KeyServiceOffering'] = data.apply(lambda x: str(x['KeyServiceOffering']), axis = 1)

In [6]:
data['Description'] = data.apply(lambda x: removeNewLine(x['Description']), axis = 1)

In [9]:
def text_process(mess):

    nopunc = [char for char in mess if char not in string.punctuation]

    nopunc = ''.join(nopunc)
    
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [8]:
data.dropna(inplace=True)


In [ ]:

data['Subject'] = data.apply(lambda x: text_process(x['Subject']), axis = 1)
data['Description'] = data.apply(lambda x: text_process(x['Description']), axis = 1)


In [ ]:

data['Subject'] = data.apply(lambda x: " ".join(x["Subject"]), axis = 1)
data['Description'] = data.apply(lambda x: " ".join(x["Description"]), axis = 1)

In [ ]:
data['newText'] = data.apply(lambda x: x["Subject"]+ " "+x["Description"], axis = 1)

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

data['newText'] = data.apply(lambda x: lemmatize_text(x['newText']), axis = 1)
data['newText'] = data.apply(lambda x: " ".join(x["newText"]), axis = 1)

In [147]:
# def removeNonEng(text):
#     words = set(nltk.corpus.words.words())
#     word2 = " ".join(w for w in nltk.wordpunct_tokenize(text) \
#              if w.lower() in words or not w.isalpha())
#     return word2

In [150]:
# data['newText'] = data.apply(lambda x: removeNonEng(x['newText']), axis = 1)

In [165]:
msg_train, msg_test, label_train, label_test = \
train_test_split(data["newText"], data['KeyCategory'], test_size=0.2)
bow_transformer = CountVectorizer(max_features=3000).fit(msg_train)


In [166]:
messages_bow = bow_transformer.transform(msg_train)
tfidf_transformer = TfidfTransformer().fit(messages_bow)
messages_tfidf = tfidf_transformer.transform(messages_bow)
# %time testmodel = GradientBoostingClassifier(learning_rate=0.01,max_depth=20, n_estimators=100,verbose=3).fit(messages_tfidf, label_train)
# testmodel = RandomForestClassifier(n_estimators=100,verbose=1,n_jobs=2).fit(messages_tfidf, label_train)
testmodel = LinearSVC().fit(messages_tfidf, label_train)

messages_bow = bow_transformer.transform(msg_test)
messages_tfidf = tfidf_transformer.transform(messages_bow)



all_predictions = testmodel.predict(messages_tfidf)


In [167]:
print (classification_report(label_test, all_predictions))

             precision    recall  f1-score   support

        1.0       0.41      0.26      0.32       104
       10.0       0.51      0.26      0.34        78
       11.0       0.75      0.50      0.60        12
       12.0       0.64      0.67      0.66       443
       13.0       0.73      0.35      0.48        31
        2.0       0.53      0.55      0.54       394
        3.0       0.67      0.57      0.62         7
        4.0       0.70      0.77      0.73       337
        5.0       0.82      0.61      0.70       161
        6.0       0.67      0.80      0.73       405
        7.0       0.70      0.67      0.68        24
        8.0       0.89      0.84      0.86       229

avg / total       0.66      0.66      0.66      2225

